In [ ]:
from pyspark.sql import SparkSession

spark = (
    SparkSession
    .builder
    .appName("Reading from sockets)
    .master("local[*]")
    .getOrCreate()
)

spark

In [ ]:
# Read input data
df_raw = spark.readStream.format("socket").options("host", "localhost").option("port", "9999").load()

In [ ]:
df_raw.printSchema()

In [ ]:
df_words = df_raw.withColumn("words", split("value", " "))

In [ ]:
# Explode the list of words
from pyspark.sql.functions import explode

df_explode = df_words.withColumn("word", explode("words")).drop("value", "words")

In [ ]:
# Aggregate the words to generate count
from pyspark.sql.functions import count, lit

df_agg = df_explode.groupBy("word").agg(count(lit(1)).alias("cnt"))

In [ ]:
# Write the output to console streaming

df_agg.writeStream.format("console").outputMode("complete").start().awaitTermination()